# **INTELIGENCIA ARTIFICIAL APLICADA A LA CIBERSEGURIDAD**
## **PRÁCTICA P3 - BLOQUE III**

**INSTRUCCIONES / RECOMENDACIONES**

- Se recomienda leer con detalle la descripción de cada una de las celdas.
- Las celdas que ya tienen código, se deberán ejecutar directamente. NOTA: algunas celdas con código deberán parametrizarse según lo indicado en cada caso.
- Las celdas que están vacías, se completarán con la implementación requerida en el notebook.
- No se incluirán más celdas de las establecidas en el presente notebook, por lo que la solución al mismo deberá implementarse exclusivamente en las celdas vacías.
- La entrega se realizará vía Moodle. Será necesario subir la solución a este notebook con el nombre: **NOMBRE_GRUPO.ipynb**

- **Fecha de Publicación: 08/04/2024**
- **Fecha de Entrega: 14/04/2024**
- **Test: 15/04/2024**

In [1]:
# SETUP

!pip3 install transformers
!pip3 install einops
!pip3 install accelerate
!pip3 install unstructured-pytesseract
!pip3 install unstructured-inference
!pip3 install sentence_transformers
!pip3 install chromadb

#!pip3 install protobuf==3.20.*

!pip3 install langchain

In [2]:
!pip3 install unstructured
!pip3 install pillow_heif
#!pip3 install cmake
!pip3 install pikepdf pypdf
#!pip3 install python-poppler

In [3]:
# IMPORTS

from transformers import AutoTokenizer
import transformers
import torch

import langchain

langchain.__version__

'0.1.16'

## LLM

Se deberá parametrizar el modelo LLM según los siguientes atributos:
- model_id
- max_length
- max_new_tokens

In [4]:
from langchain import HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    model_kwargs={
        "max_length":1024,
        'do_sample': True,
        'top_k': 10,
        'num_return_sequences': 2,
        #'device_map': 'auto',
        'trust_remote_code': True,
        'torch_dtype': torch.bfloat16
    },
    pipeline_kwargs={"max_new_tokens": 500},
    device=0,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## DOCUMENTS & SPLITTER & VECTORSTORE

Se deberá construir un sistema RAG sobre un caso de uso de Ciberseguridad, utilizando para ello el modelo LLM especificado. La selección del caso de uso de Ciberseguridad será propuesta por el alumn@, estableciendo el documento PDF base (**pdf_base**) para realizar posteriormente las consultas al modelo.

In [5]:
# INDEXING

from langchain.document_loaders import OnlinePDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain

In [6]:
# 1. LOAD
pdf_base = "https://unece.org/sites/default/files/2023-02/R155e%20%282%29.pdf"
loader = OnlinePDFLoader(pdf_base)
document = loader.load()

# 2. SPLIT
text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=64)
documents = text_splitter.split_documents(document)

# 3. EMBED & STORE
embeddings = HuggingFaceEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings)

# RETRIEVAL & GENERATION
qa = ConversationalRetrievalChain.from_llm(
    llm,
    vectorstore.as_retriever(),
    return_source_documents=True,
)

## QUESTION TO LLM = MY_QUERY + CONTEXT_from_VECTORSTORE -> ANSWER FROM LLM

Se deberá generar un catálogo de 10 prompts de consultas sobre el sistema RAG generado, utilizando para ello la variable **my_query**. Los prompts generados deberán obtener respuestas del modelo LLM con una calidad adecuada.

In [7]:
# QUESTION TO LLM

import warnings
warnings.filterwarnings('ignore')

chat_history = []

In [8]:
my_query ="What is the specific process to certify a car?"
result = qa({"question": my_query, "chat_history": chat_history})
print(result["answer"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

5.1.1.

The Approval Authority or the Technical Service shall verify by means of document checks that the vehicle manufacturer has taken the necessary measures relevant for the vehicle type to:

(a)

Collect and verify the information required under this Regulation through the supply chain so as to demonstrate that supplier-related risks are identified and are managed;

6.2.

An application for a Certificate of Compliance for Cyber Security Management System shall be submitted by the vehicle manufacturer or by their duly accredited representative.

6.3.

It shall be accompanied by the undermentioned documents in triplicate, and by the following particular:

6.3.1.

Documents describing the Cyber Security Management System.

6.3.2.

A signed declaration using the model as defined in Appendix 1 to Annex 1.

6.4.

5.1.2.

The A

In [9]:
my_query ="Do you have to certify every car only one per model?"
result = qa({"question": my_query, "chat_history": chat_history})
print(result["answer"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

7.2.2.5.

The vehicle manufacturer shall be required to demonstrate how their Cyber Security Management System will manage dependencies that may exist with contracted suppliers, service providers or manufacturer’s sub-organizations in regards of the requirements of paragraph 7.2.2.2.

7.3.

Requirements for vehicle types

7.3.1.

The manufacturer shall have a valid Certificate of Compliance for the Cyber Security Management System relevant to the vehicle type being approved.

5.1.1.

The Approval Authority or the Technical Service shall verify by means of document checks that the vehicle manufacturer has taken the necessary measures relevant for the vehicle type to:

(a)

Collect and verify the information required under this Regulation through the supply chain so as to demonstrate that supplier-related risks are identified 

# Conclusiones

Detalle las conclusiones extraídas del presente estudio sobre LLMs y sistemas RAG.

*# Escriba aquí las conclusiones*